<a href="https://colab.research.google.com/github/GWANGHYUNYU/pytorch-tutorial/blob/master/08_1_xor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
def get_AND_data():
    X = [[0, 0], [0, 1], [1, 0], [1, 1]]
    Y = [[0],[0],[0],[1]]
    return X, Y

def get_OR_data():
    X = [[0, 0], [0, 1], [1, 0], [1, 1]]
    Y = [[0],[1],[1],[1]]
    return X, Y

def get_XOR_data():
    X = [[0, 0], [0, 1], [1, 0], [1, 1]]
    Y = [[0], [1], [1], [0]]
    return X, Y

In [ ]:
# X, Y = get_AND_data()
# X, Y = get_OR_data()
X, Y = get_XOR_data()

In [ ]:
x_train = torch.FloatTensor(X)
y_train = torch.FloatTensor(Y)

In [ ]:
print(x_train.shape)
print(y_train.shape)

torch.Size([4, 2])
torch.Size([4, 1])


In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [ ]:
model = Classifier()

In [ ]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)
    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/100 Cost: 0.727397 Accuracy 50.00%
Epoch   10/100 Cost: 0.695116 Accuracy 50.00%
Epoch   20/100 Cost: 0.693778 Accuracy 50.00%
Epoch   30/100 Cost: 0.693361 Accuracy 50.00%
Epoch   40/100 Cost: 0.693224 Accuracy 50.00%
Epoch   50/100 Cost: 0.693176 Accuracy 50.00%
Epoch   60/100 Cost: 0.693159 Accuracy 50.00%
Epoch   70/100 Cost: 0.693152 Accuracy 50.00%
Epoch   80/100 Cost: 0.693149 Accuracy 50.00%
Epoch   90/100 Cost: 0.693148 Accuracy 50.00%
Epoch  100/100 Cost: 0.693148 Accuracy 50.00%


In [ ]:
# Accuracy computation
# True if hypothesis>0.5 else False
with torch.no_grad():
    hypothesis = model(x_train)
    # print(type(hypothesis))
    predicted = (hypothesis > 0.5)
    # print(predicted)
    correct_prediction = (predicted == y_train)
    # print(correct_prediction, type(correct_prediction))
    accuracy = correct_prediction.sum().item() / len(correct_prediction)
    # print(accuracy)
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy)


Hypothesis:  [[0.50067806]
 [0.5002141 ]
 [0.4999987 ]
 [0.49953476]] 
Correct:  [[ True]
 [ True]
 [False]
 [False]] 
Accuracy:  0.5


XOR 문제를 해결하기 위한 MLP

In [ ]:
X, Y = get_XOR_data()

In [ ]:
print(X, Y)

[[0, 0], [0, 1], [1, 0], [1, 1]] [[0], [1], [1], [0]]


In [ ]:
x_train = torch.FloatTensor(X)
y_train = torch.FloatTensor(Y)

XOR 모델을 torch.nn.Sequential 로 만드는 방법

In [ ]:
# nn layers
linear1 = torch.nn.Linear(2, 2, bias=True)
linear2 = torch.nn.Linear(2, 1, bias=True)
sigmoid = torch.nn.Sigmoid()

# model
model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid).to(device)

XOR 모델을 torch.nn.Module 로 만드는 방법

In [ ]:
class XOR_Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(2, 2, bias=True)
        self.linear2 = nn.Linear(2, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.linear1(x))
        x = self.sigmoid(self.linear2(x))
        return x

In [ ]:
model = XOR_Classifier()

In [ ]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)
    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/1000 Cost: 0.741045 Accuracy 50.00%
Epoch  100/1000 Cost: 0.691637 Accuracy 50.00%
Epoch  200/1000 Cost: 0.682888 Accuracy 75.00%
Epoch  300/1000 Cost: 0.619116 Accuracy 75.00%
Epoch  400/1000 Cost: 0.445046 Accuracy 75.00%
Epoch  500/1000 Cost: 0.164295 Accuracy 100.00%
Epoch  600/1000 Cost: 0.072427 Accuracy 100.00%
Epoch  700/1000 Cost: 0.043958 Accuracy 100.00%
Epoch  800/1000 Cost: 0.031071 Accuracy 100.00%
Epoch  900/1000 Cost: 0.023870 Accuracy 100.00%
Epoch 1000/1000 Cost: 0.019313 Accuracy 100.00%


In [ ]:
# Accuracy computation
# True if hypothesis>0.5 else False
with torch.no_grad():
    hypothesis = model(x_train)
    # print(type(hypothesis))
    predicted = (hypothesis > 0.5)
    # print(predicted)
    correct_prediction = (predicted == y_train)
    # print(correct_prediction, type(correct_prediction))
    accuracy = correct_prediction.sum().item() / len(correct_prediction)
    # print(accuracy)
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy)


Hypothesis:  [[0.02211061]
 [0.9825607 ]
 [0.98252976]
 [0.01933792]] 
Correct:  [[False]
 [ True]
 [ True]
 [False]] 
Accuracy:  1.0


MLP

In [ ]:
X, Y = get_XOR_data()

In [ ]:
x_train = torch.FloatTensor(X)
y_train = torch.FloatTensor(Y)

In [ ]:
# nn layers
linear1 = torch.nn.Linear(2, 10, bias=True)
linear2 = torch.nn.Linear(10, 10, bias=True)
linear3 = torch.nn.Linear(10, 10, bias=True)
linear4 = torch.nn.Linear(10, 1, bias=True)
sigmoid = torch.nn.Sigmoid()

In [ ]:
# model
model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid, linear3, sigmoid, linear4, sigmoid).to(device)

In [ ]:
# define cost/loss & optimizer
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)  # modified learning rate from 0.1 to 1

In [ ]:
for step in range(10001):
    optimizer.zero_grad()
    hypothesis = model(x_train)

    # cost/loss function
    cost = criterion(hypothesis, y_train)
    cost.backward()
    optimizer.step()

    if step % 100 == 0:
        print(step, cost.item())

0 0.7408947944641113
100 0.6931389570236206
200 0.6931374669075012
300 0.6931357383728027
400 0.6931341886520386
500 0.6931324601173401
600 0.693130612373352
700 0.6931287050247192
800 0.6931267976760864
900 0.6931246519088745
1000 0.6931225061416626
1100 0.693120002746582
1200 0.6931174993515015
1300 0.6931147575378418
1400 0.693111777305603
1500 0.6931085586547852
1600 0.6931051015853882
1700 0.6931012868881226
1800 0.6930969953536987
1900 0.6930922269821167
2000 0.6930870413780212
2100 0.693081259727478
2200 0.693074643611908
2300 0.6930671334266663
2400 0.6930586099624634
2500 0.6930486559867859
2600 0.693037211894989
2700 0.6930238008499146
2800 0.6930078268051147
2900 0.6929886937141418
3000 0.6929655075073242
3100 0.6929367780685425
3200 0.6929006576538086
3300 0.6928543448448181
3400 0.6927931904792786
3500 0.6927102208137512
3600 0.6925929188728333
3700 0.6924187541007996
3800 0.6921424269676208
3900 0.6916618347167969
4000 0.6907041668891907
4100 0.6883158683776855
4200 0.679

In [ ]:
# Accuracy computation
# True if hypothesis>0.5 else False
with torch.no_grad():
    hypothesis = model(x_train)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == y_train).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy.item())


Hypothesis:  [[6.8877263e-05]
 [9.9988139e-01]
 [9.9989223e-01]
 [1.5683484e-04]] 
Correct:  [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:  1.0


In [ ]:
class MLP_Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(2, 10, bias=True)
        self.linear2 = nn.Linear(10, 10, bias=True)
        self.linear3 = nn.Linear(10, 5, bias=True)
        self.linear4 = nn.Linear(5, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.linear1(x))
        x = self.sigmoid(self.linear2(x))
        x = self.sigmoid(self.linear3(x))
        x = self.sigmoid(self.linear4(x))
        return x

In [ ]:
model = MLP_Classifier()

In [ ]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 10001
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)
    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/10001 Cost: 0.693147 Accuracy 50.00%
Epoch  100/10001 Cost: 0.693147 Accuracy 50.00%
Epoch  200/10001 Cost: 0.693146 Accuracy 50.00%
Epoch  300/10001 Cost: 0.693146 Accuracy 50.00%
Epoch  400/10001 Cost: 0.693146 Accuracy 50.00%
Epoch  500/10001 Cost: 0.693145 Accuracy 50.00%
Epoch  600/10001 Cost: 0.693145 Accuracy 50.00%
Epoch  700/10001 Cost: 0.693144 Accuracy 50.00%
Epoch  800/10001 Cost: 0.693144 Accuracy 50.00%
Epoch  900/10001 Cost: 0.693143 Accuracy 50.00%
Epoch 1000/10001 Cost: 0.693143 Accuracy 50.00%
Epoch 1100/10001 Cost: 0.693142 Accuracy 50.00%
Epoch 1200/10001 Cost: 0.693142 Accuracy 50.00%
Epoch 1300/10001 Cost: 0.693142 Accuracy 50.00%
Epoch 1400/10001 Cost: 0.693141 Accuracy 50.00%
Epoch 1500/10001 Cost: 0.693141 Accuracy 50.00%
Epoch 1600/10001 Cost: 0.693140 Accuracy 50.00%
Epoch 1700/10001 Cost: 0.693139 Accuracy 50.00%
Epoch 1800/10001 Cost: 0.693139 Accuracy 50.00%
Epoch 1900/10001 Cost: 0.693138 Accuracy 50.00%
Epoch 2000/10001 Cost: 0.693138 Accuracy

In [ ]:
# Accuracy computation
# True if hypothesis>0.5 else False
with torch.no_grad():
    hypothesis = model(x_train)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == y_train).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy.item())


Hypothesis:  [[4.5305054e-04]
 [9.9956638e-01]
 [9.9949956e-01]
 [4.6742201e-04]] 
Correct:  [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:  1.0
